# 7. 扩充任务（Expanding）

扩充任务，是将一小段简短的文本，例如一组说明或主题列表，用大型语言模型生成一段更长的文本，例如关于某个主题的电子邮件或一篇文章。

这有一些很好的用途，例如你可以将大型语言模型用作头脑风暴的合作伙伴。但是我也要承认，这方面存在一些有问题的使用案例，例如如果有人使用它产生大量垃圾邮件。因此，当你使用大型语言模型的这些能力时，请以负责任的方式来使用，以有助于人的方式来使用。

在这个视频中，我们将通过一个示例说明，如何使用语言模型基于一些信息生成个性化的电子邮件。这封电子邮件自称来自一个 AI 机器人，正如 Andrew（吴恩达）所说的，这非常重要。

我们还将使用另一个模型的输入参数，称为温度（temperature），该参数允许你改变模型探索和多样性的程度。让我们开始吧。

## 7.1 AI 自动回复邮件

在我们开始之前，我们要做一些常规的设置。设置 OpenAI Python包，然后定义辅助函数 get_completion()。

In [ ]:
import openai
import os
from openai import OpenAI

# 1. 根据环境变量获取 openai key
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY') 

client = OpenAI()

# 2. 定义 get_completion 方法
def get_completion(prompt, instructions=None, model="gpt-3.5-turbo", temperature=0):
    response = client.responses.create(
        model=model,
        instructions=instructions,
        input=prompt,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.output_text

现在我们要为客户的评论，写一个自定义电子邮件回复，因此，针对一条客户评论和情绪，我们将生成一个自定义的回复。

现在，我们将使用语言模型，根据客户的评论和评论的情绪，给客户发送一封定制的的电子邮件。

这是搅拌机的客户评论，我们已经使用在推理任务视频中看到的那种提示提取了评论的情绪，现在我们将根据评论的情绪定制回复。

因此，这里的提示是：你是一名客户服务的 AI 助理，你的任务给客户发送一封电子邮件回复，以感谢客户的评论。提示以三个反引号```进行分隔。如果情绪是积极的或中立的，感谢他们的评论；如果情绪是负面的，请道歉，并建议他们可以联系客户服务部门。回复要确保使用评论中的具体细节，以简洁和专业的语气写作，并以 AI 客户代理的身份在电子邮件中签名。当你使用语言模型生成给用户的文本时，这种透明度是非常重要的。让用户知道他们看到的文本是由 AI 生成的，这一点非常重要。

然后我们输入客户的评论和评论的情绪。是否输入评论的情绪并不重要，因为我们实际上可以使用这个提示来提取评论的情绪，然后在后续步骤中再编写电子邮件。但这里只是举例，就假定我们已经从评论中提取了情绪。

In [ ]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""

response = get_completion(prompt)
print(response) 

于是，生成了一个给客户的回复，它涉及客户在评论中提到的细节。正如我们所指示的，建议客户联系客户服务，因为这只是一个 AI 客户服务代理。

Subject: Apology and Resolution for Your Recent Experience

Dear Valued Customer,

Thank you for taking the time to share your detailed feedback with us. We sincerely apologize for the inconvenience you experienced with the pricing changes and the quality of the product you purchased. Your insights are valuable to us, and we are committed to improving our products and services based on customer feedback.

We regret to hear about the issue with the motor and the warranty expiration. We understand your frustration and would like to offer our assistance. Please feel free to reach out to our customer service team at [customer service contact information] for further support and resolution.

We appreciate your loyalty and trust in our brand. Your feedback will be shared with our product development team to ensure better quality control in the future.

Thank you once again for bringing this to our attention. We value your feedback and strive to provide a better experience for all our customers.

AI Customer Agent

## 7.2 温度参数的影响

接下来，我们将使用语言模型参数的一个参数，称为温度（temperature）。它将允许我们能改变模型响应的多样性。你可以把温度看作是模型的探索或随机性的程度。

对于这个特定的短语，“我最喜欢的食物是……”，模型预测最有可能的下一个单词是披萨，其次最有可能的单词是寿司和墨西哥卷饼。因此，在 temperature=0 时，模型总是会选择最有可能的下一个单词，在这种情况下是披萨。而在更高的温度参数下，它也可能会选择一个不是最大概率的单词。在更高温度时，甚至可能选择玉米卷，虽然只有 5% 的概率被选中。

![temperature](./img/7-1.jpg)

你可以想象，当模型继续这个最后的响应时，“我最喜欢的食物是披萨”，而且它会继续产生更多的单词，这个响应与第一个响应“我最爱的食物是玉米卷”发生偏离。因此，随着模型继续产生更多内容，这两种响应将变得越来越不同。一般来说，当构建应用程序时，如果需要得到可预测的模型响应，我建议设置 temperature=0。

在本课程的视频中，我们一直在使用 temperature=0。我认为如果你试图建立一个可靠的和可预测的系统，你就应该使用 temperature=0。而如果你希望模型更有创造性，你可能想获得更加多样性的不同输出，你可能需要使用更高的温度参数。

那么，现在让我们使用刚才使用的相同提示，并尝试生成一封电子邮件，但让我们使用更高的温度参数。我们在视频中一直使用的 get_completion 函数中，已经指定了模型和温度参数，但我们此前是将其设置为默认值。现在，让我们试着改变温度值。

我们使用提示，然后让我们试试 temperature=0.7。

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""

response = get_completion(prompt, temperature=0.7)
print(response) 

在 temperature=0 的情况下，每次执行相同的提示时，你可以期待相同的输出。而当 temperature=0.7 时，每次都会得到不同的输出。

Subject: Apology and Assistance Regarding Your Recent Experience

Dear valued customer,

Thank you for taking the time to share your feedback with us. We are truly sorry to hear about the challenges you encountered with our pricing and product quality. Your detailed insights are greatly appreciated, and we apologize for any inconvenience this may have caused.

We strive to provide our customers with the best value and quality products, and your feedback helps us identify areas where we can improve. If you have any further concerns or would like to discuss this matter further, please do not hesitate to reach out to our customer service team for assistance.

We value your loyalty and trust in our brand, and we are committed to addressing your feedback to ensure a better experience in the future.

Thank you once again for bringing this to our attention.

AI customer agent

这里输出了生成的电子邮件，正如你所看到的，这与我们以前生成的电子邮件不同。让我们再执行一次，将又一次得到不同的电子邮件。

我建议你自己尝试一下，改变温度参数的值。你现在可以暂停视频，在各种不同的温度值尝试这个提示，看看模型的输出是如何变化的。

总之，在较高的温度值下，模型的输出将更加随机。你可以认为，在更高的温度下，AI 助理更容易分心，但也许更加具有创造力。

在下一个视频中，我们将更多地讨论聊天完成端点格式（ Chat Completions Endpoint format ），以及如何使用这种格式创建一个自定义的聊天机器人。